In [10]:
import numpy as np
import csv
import math

In [11]:
x = np.loadtxt(open("mnist_train.csv", "rb"), delimiter=",", skiprows=1)

train_labels = x[:50000, 0]
train_data = x[:50000, 1:]

valid_labels = x[50000:57000, 0]
valid_data = x[50000:57000, 1:]

test_labels = x[57000:,0]
test_data = x[57000:,1:]

unseen_data = np.loadtxt(open("mnist_test.csv", "rb"), delimiter=",", skiprows=1)



In [189]:
def relu(x):
    
    return x*np.int64(x>0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Th
def softmax(x):
    #print('avant: ', x)
    """Compute softmax values for each sets of scores in x."""
    #e_x = np.exp(x - np.max(x))
    return np.exp(x) / np.sum(np.exp(x), axis=0)



def non_linearity(x, nonlin='relu'):
    if nonlin == 'relu':
        return x*np.int64(x>0)
    elif nonlin == 'sigmoid':
        return 1 / (1 + np.exp(-1*x))
    elif nonlin == 'softmax':
        exps = np.exp(x - np.max(x))
        return exps / exps.sum(axis=0)
        
#//////////////////////////////////////////////////////////////////
def rand_index(upper_bound):
    return np.random.randint(0, upper_bound)


#////////////////////////////////////////////////////////////////////

from sklearn.metrics import log_loss
def cross_entropy(predictions, targets):
    target = np.eye(10, dtype=int)[int(targets)]
    ce = -np.sum(target*np.log(predictions))
    return ce


# where y is true (1 or 0) and i is the class
def loss_one_example(os, y, i):
    
    return -np.log(os[y,i])

def cross_entr_loss(prediction, true_label):
    # Calculate cross entropy for each output vector element , then sum up
    loss = 0 
    y = np.eye(10)[int(true_label)]

    for k,elem in enumerate(y):
            loss += loss_one_example(prediction[y], elem ,k)
    return loss



def CrossEntropy(yHat, y):
    if yHat == 1:
        return -np.log(y)
    else:
        return -np.log(1 - y)
    

In [221]:
# Three different ways to initialize parameters

def initialize_weights( in_dim, out_dim, technique='glorot', sigma=0.002, mu=0):
        # Init with zeros for now
        if technique =='zeros':
            return np.zeros((in_dim,out_dim))
        elif technique == 'normal':
            return sigma * np.random.randn(in_dim, out_dim) + mu
        elif technique == 'glorot':
            return np.random.randn(in_dim, out_dim) * np.sqrt(1/out_dim)

In [226]:
# Sets of weights are: w_1 and b1 for layer 1, up to layer 3
# pre actiavation neuron outputs are referred to as h1_a for layer 1
# post activation neuron output are h1_o for layer 1
# Note that I note h3 instead of a more common ho for the output layer


class MLP:
    
    def __init__(self, hidden_dims=(200,600),n_hidden=2, step_size = 0.0001, train_data=train_data,
                 train_labels=train_labels, valid_data=valid_data, valid_labels=valid_labels, classes=10):
        
        
        self.train_data = train_data
        self.train_labels = train_labels
        self.valid_data = valid_data
        self.valid_labels = valid_labels
        
        self.hidden_dims = hidden_dims
        self.input_dim = train_data.shape[1]
        self.classes = classes
        
        self.w_1 = initialize_weights(hidden_dims[0], self.input_dim)
        self.w_2 = initialize_weights(hidden_dims[0],hidden_dims[1])        
        self.w_3 = initialize_weights(classes, hidden_dims[1])
        
        self.b1 = np.zeros(hidden_dims[0])
        self.b2 = np.zeros(hidden_dims[1])
        self.b3 = np.zeros(classes)
        
 
            
    def forward_prop(self, x, non_lin='relu'):
        # Simply passes forward on input vector
        # return output pre and post activation at each layer
        h1_a = np.matmul(self.w_1, x) + self.b1 
        #print('h1_a: ', h1_a.shape)

        h1_o = non_linearity(h1_a, non_lin)
        #print('h1_0: ', h1_o.shape)
        h2_a = np.matmul(np.transpose(self.w_2) , h1_o) + self.b2 
        #print('h2_a: ', h2_a.shape)
        h2_o = non_linearity(h2_a, non_lin)
        #print('h2_0: ', h2_o.shape)
        h3_a = np.matmul(self.w_3 , h2_o) + self.b3 
        h3_o = softmax(h3_a)
        
        return (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o)  
    
    def backward_prop(self, h1_a, h1_o, h2_a, h2_o, h3_a, h3_o, inputs, labels):
        
        # Find gradient of loss with respect to each parameters
        
        stuff = np.eye(self.classes)[int(labels)].reshape(self.classes,)
        grad_h3a = h3_o - stuff
        grad_b3 = grad_h3a
        grad_W3 = np.matmul(grad_h3a, np.transpose(h3_o))
        
        grad_h2o = np.matmul(np.transpose(self.w_3), grad_h3a)
        grad_h2a = grad_h2o * np.int64(h2_a>0)
        grad_b2 = grad_h2a
        grad_W2 = np.matmul(grad_h2a, np.transpose(h1_o))
        
        grad_h1o = np.matmul(np.transpose(self.w_2), grad_h2a)
        grad_h1a = grad_h1o * np.int64(h1_a > 0)
        grad_b1 = grad_h1a
        grad_W1 = np.matmul(grad_h1a.reshape(500,1), np.transpose(inputs.reshape(784,1)))
                
        return (grad_W1, grad_b1, grad_W2, grad_b2, grad_W3, grad_b3)
    
    # A function that trains a whole minibatch and average the gradients, to be sent for parameter update
    def train_minibatch(self, batch_size):
        # fprop and bprop for batch_size inputs, each time summing grads
        
        sum_grad_W1 = np.zeros((self.w_1.shape[0], self.w_1.shape[1]))
        sum_grad_W2 = np.zeros((self.w_2.shape[0], self.w_2.shape[1]))
        sum_grad_W3 = np.zeros((self.w_3.shape[0], self.w_3.shape[1]))
        sum_grad_b1 = np.zeros(self.b1.shape[0])
        sum_grad_b2 = np.zeros(self.b2.shape[0])
        sum_grad_b3 = np.zeros(self.b3.shape[0])

        
        for i in range(batch_size):
            z = rand_index(self.train_data.shape[0])
            (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o)  = self.forward_prop(train_data[z,:])
            (grad_W1, grad_b1, grad_W2, grad_b2, grad_W3, grad_b3) = self.backward_prop(h1_a, h1_o, h2_a, h2_o, h3_a, h3_o, train_data[z,:] , train_labels[z] )
            sum_grad_W1 = np.add(sum_grad_W1, grad_W1)
            sum_grad_b1 = np.add(sum_grad_b1, grad_b1)
            sum_grad_W2 = np.add(sum_grad_W2, grad_W2)
            sum_grad_b2 = np.add(sum_grad_b2, grad_b2)
            sum_grad_W3 = np.add(sum_grad_W3, grad_W3)
            sum_grad_b3 = np.add(sum_grad_b3, grad_b3)
        # return average of each gradients over the batch
        return (sum_grad_W1/batch_size, sum_grad_b1/batch_size, sum_grad_W2/batch_size, sum_grad_b2/batch_size, 
                sum_grad_W3/batch_size, sum_grad_b3/batch_size)
            
    
    
    #Now that we have gradients, build a function to update all values using those gradient
    def update_params(self, grad_W1, grad_b1, grad_W2, grad_b2, grad_W3, grad_b3, step_size):
        self.w_1 = self.w_1 - (step_size * grad_W1 )
        self.b1 = self.b1 - (step_size * grad_b1)
        self.w_2 = self.w_2 - (step_size * grad_W2)
        self.b2 = self.b2 - (step_size * grad_b2)
        self.w_3 = self.w_3 - (step_size * grad_W3)
        self.b3 = self.b3 - (step_size * grad_b3)
        
    # Full training, for n epochs
    def train(self, batch_size, epochs, step_size):
        for j in range(epochs):
            sum_loss = 0
            (grad_w1, grad_b1, grad_w2, grad_b2, grad_w3, grad_b3 ) = self.train_minibatch(batch_size)
            self.update_params(grad_w1, grad_b1, grad_w2, grad_b2, grad_w3, grad_b3, step_size)
            # after updating, calculate and output average loss
            sum_loss = self.average_train_loss()
            print('Average loss on training set after ',j+1 ,' epochs: ', sum_loss)
            #print('Misclassification: ', self.test_misclass(train_data, train_labels), '%')
            #print('-----------------------') 

        #then calculate loss on validation set
        valid_loss = self.average_valid_loss()
        print('Loss on validation set after ', (epochs+1),' epochs of training (with step size = ', step_size,'): ', valid_loss)
        
        print('Misclassification: ', self.test_misclass(valid_data, valid_labels), '%')
        print('-----------------------') 
        
        
    # prediction function for 1 input, with current mlp parameters values
    def predict(self, inputs):
        x = np.matmul(self.w_1, inputs) + self.b1
        x = relu(x)
        x = np.matmul(self.w_2, x) + self.b2
        x = relu(x)
        x = np.matmul(self.w_3, x) + self.b3
        output = softmax(x)
        
        return argmax(output)
    
    # calculate average loss over whole train set
    def average_train_loss(self):
        loss = 0
        # summing loss
        for k in range(self.train_data.shape[0]):
            randindex = rand_index(self.train_data.shape[0])
            (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o) = self.forward_prop(train_data[randindex,...])    
            one_loss = np.sum(cross_entropy(h3_o, train_labels[randindex]))
            loss += one_loss
            
        return loss/self.train_data.shape[0]
        
    def average_valid_loss(self):
        loss = 0
        # summing loss
        for k in range(self.valid_data.shape[0]):
            randindex = rand_index(self.valid_data.shape[0])
            (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o) = self.forward_prop(valid_data[randindex,...])    
            one_loss = np.sum(cross_entropy(h3_o, valid_labels[randindex]))
            loss += one_loss
        return (loss/self.train_data.shape[0])
    
    def test_misclass(self, data, labels):
        misclass = 0
        
        for k in range(data.shape[0]):
            (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o) = self.forward_prop(data[k,:])
            pred = np.argmax(h3_o)
            if(pred != int(labels[k])):
                misclass += 1
        
        return ((misclass/data.shape[0])*100 )
        
        

In [227]:
mlp = MLP()
print(mlp)

In [209]:
# pred output --> (h1_a, h1_o, h2_a, h2_o, h3_a, h3_o) 
pred = mlp.forward_prop(train_data[4,...])
output_vect = pred[5]
print('output vector: ', output_vect)

#loss_sum = cross_entropy(output_vect, int(train_labels[4]) )
#print(np.sum(loss_sum))

output vector:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [204]:


#self, h1_a, h1_o, h2_a, h2_o, h3_a, h3_o, inputs, labels
#(grad_W1, grad_b1, grad_W2, grad_b2, grad_W3, grad_b3) = mlp.backward_prop(pred[0], pred[1], pred[2], pred[3], pred[4], pred[5], train_data[1,...], np.argmax(train_labels[4]))

#print(grad_W1.shape)

In [223]:
#def train(self, batch_size, epochs, step_size):
mlp2 = MLP()
mlp2.train(128, 10, 1)

Average loss on training set after  1  epochs:  2.318167632733662
Average loss on training set after  2  epochs:  2.317117969603752
Average loss on training set after  3  epochs:  2.3117237044459706
Average loss on training set after  4  epochs:  2.3128296617430633
Average loss on training set after  5  epochs:  2.3115271142787863
Average loss on training set after  6  epochs:  2.3114278464696123
Average loss on training set after  7  epochs:  2.310222790445528
Average loss on training set after  8  epochs:  2.309907272840814
Average loss on training set after  9  epochs:  2.309838919071448
Average loss on training set after  10  epochs:  2.3077291211154622
Loss on validation set after  11  epochs of training (with step size =  1 ):  0.32306161328940874
Misclassification:  90.3 %
-----------------------


In [228]:
#def train(self, batch_size, epochs, step_size):
mlp3 = MLP()
mlp3.train(128, 2, 0.001)

ValueError: shapes (600,) and (200,) not aligned: 600 (dim 0) != 200 (dim 0)

## Testing Set 

In [11]:
# calculate 1-0 loss on test seen
mlp2.test_misclass(test_data, test_labels)

Misclassification rate on test set:  90.4 %


In [131]:
# This was taken from my Homework 3 of Ioannis' Machine Learning class in Automn '18'
def finite_diff(self, inputs, labels):
        #for each parameters we vary by epsilon one of its value and compute the finite gradient
        #(f(x-eps) - f(x+eps))/eps
        epsilon = 10**(-4)
        grads_finite = []
        for i,mat in enumerate([self.W1, self.b1, self.W2, self.b2]):
            temp_mat_grad = np.zeros((len(mat), len(mat[0])))
            for row in range(len(mat)):
                for col in range(len(mat[0])):
                    W1 = np.copy(self.W1)
                    b1 = np.copy(self.b1)
                    W2 = np.copy(self.W2)
                    b2 = np.copy(self.b2)
                    l = [W1, b1, W2, b2]
                    l[i][row,col] += epsilon
                    (ha, hs, oa, os_plus_epsilon) = self.fprop(inputs, l[0], l[1], l[2], l[3])
                    l[i][row,col] -= 2* epsilon
                    (ha, hs, oa, os_minus_epsilon) = self.fprop(inputs, l[0], l[1], l[2], l[3])
                    diff = 0
                    if(type(labels)==numpy.int64):
                        diff += (self.loss_one_example(os_plus_epsilon, labels,0) - self.loss_one_example(os_minus_epsilon, labels,0)) / (2* epsilon)
                    else:
                        for k,elem in enumerate(labels):
                            diff += (self.loss_one_example(os_plus_epsilon, elem, k) - self.loss_one_example(os_minus_epsilon, elem, k)) / (2* epsilon)
                    temp_mat_grad[row,col] = diff/ inputs.shape[1]
            grads_finite.append(temp_mat_grad)
        return grads_finite